# Import Libraries

In [5]:
import pandas as pd
import numpy as np
import httpx
from selectolax.parser import HTMLParser
import re

# Get Data From vlr.gg

## Data Extracting & Cleaning Function

In [54]:
def extract_df(df):
    
    
    def extract_data(data) -> str:
        list_data = re.findall("[\d+-.]+",data)

        if len(list_data) < 3:
            result = '0 0 0'
        else:
            result = ' '.join(list_data)

        return result
    
    filled_df =df.fillna('Nan Nan Nan').astype('object')
    
    name = filled_df.iloc[:,0].map(lambda s : s.split(' ')[0])
    team = filled_df.iloc[:,0].map(lambda s : s.split(' ')[1])
    
    filled_df.iloc[:,0] = name
    filled_df.iloc[:,1] = team
    
    formatted_df = filled_df.rename(columns={'Unnamed: 0':'Name','Unnamed: 1':'Team'})
    
    formatted_df.iloc[:,2:] = formatted_df.iloc[:,2:].applymap(extract_data)
    
    new_columns = []
    for c in formatted_df.columns[2:]:
        for side in ['all','atk','def']:
            new_columns.append((c,side))

    new_columns = np.array(new_columns).reshape(-1,3,2)
    
    
    result_df = pd.DataFrame(formatted_df.iloc[:,:2])
    
    
    result_df.columns = [('Name','Name'),('Team','Team')]
    
    
    for nc,c in zip(new_columns,formatted_df.iloc[:,2:].columns):
        result_df[list(map(lambda x: tuple(x),list(nc)))] = formatted_df[c].str.split(' ',expand=True)
        
    
    result_df.columns = pd.MultiIndex.from_tuples(result_df.columns,name=['Type','Side'])
    
    return result_df

## Scraping Data

### Preparing URL(s)

In [55]:
base_url = 'https://www.vlr.gg'

event_url = '/event/matches/1188/champions-tour-2023-lock-in-s-o-paulo/?series_id=all'

### Scraping All Matches in the Event

In [56]:
res = httpx.get(base_url+event_url) #get https response from vlr.gg

html = HTMLParser(res.text) # parsing into plain text

matches = html.css('a.wf-module-item') # get all matches in events

completed_matches = [match for match in matches if match.css_first('div.ml-status').text() == 'Completed'] # find all completed match by css status class

In [57]:
# incase need a perfect matches with no missing data point(s) for VCT LOCK//IN
# perfect_completed_matches = completed_matches[:21] + completed_matches[22:30] + completed_matches[31:]

### Scraping Map(s) played and players data

In [73]:
re_strip = lambda sp,st : sp.join(re.findall('\S+',st))

for index,match in enumerate(completed_matches[:2]):
    
    # formatting name (tbh i couldn't remember why i did this but for the pretty formatted name)
    match_res = httpx.get(match_url := f'{base_url}{match.attributes["href"]}')

    match_html = HTMLParser(match_res.text)

    match_name = match_html.css_first('title').text().strip().split(' | ')[0].split(' vs. ')

    messy_match_result = match_html.css_first('div.js-spoiler').text()
    match_result = ' ' + re_strip('',messy_match_result) + ' '

    print(index, '>> ',end='')
    print(match_sum:=match_result.join(match_name))
    print()
    
    date = match_html.css_first('div.match-header-date')
    print('\t'+re_strip(' ',date.text()))
    print()
    
    stage = match_html.css_first('div.match-header-event-series')
    print('\t'+re_strip(' ',stage.text()))
    print()
    
    ban_pick = match_html.css_first('div.match-header-note')
    for bp in re_strip(' ',ban_pick.text()).split(';'):
        print('\t'+bp.strip())
#     #scraping all tables from pages w/ css table class
#     tables = match_html.css('table.wf-table-inset')

    
#     for table in tables: # for all table in matches
#         df = pd.read_html(table.html)[0]
#         display(transform_subcolumns_df(df))


    maps = match_html.css('div.vm-stats-game')

    maps = [maps[1],maps[0],maps[2]]
    
    for m in maps:
        current_map = m.css_first('div.map')
        match_header = 'Overall' if not current_map else ' '.join(re_strip(' ',current_map.text()).split(' ')[::2])
        
        print(match_header)
        
        tables = m.css('table.wf-table-inset')
        
        for table in tables:
            df = pd.read_html(table.html)[0]
            display(extract_df(df))
        

0 >> KOI 0:2 NRG Esports

	Tuesday, February 14th 12:10 AM +07 Patch 6.02

	Bracket Stage: Alpha - Round of 16

	NRG ban Ascent
	KOI ban Split
	NRG pick Icebox
	KOI pick Haven
	NRG ban Fracture
	KOI ban Lotus
	Pearl remains
Overall


Type        Name Team     R              ACS             K      ... HS%  FK  \
Side        Name Team   all   atk   def  all  atk  def all atk  ... def all   
0          trexx  KOI  1.24  1.45  1.03  275  329  229  46  28  ...  37   3   
1        sheydos  KOI  0.95  1.16  0.75  158  188  135  27  18  ...  25   2   
2         starxo  KOI  0.76  0.79  0.72  123  123  124  17   9  ...  13   0   
3         Wolfen  KOI  0.75  0.81  0.70  187  214  161  28  16  ...  40  10   
4     koldamenta  KOI  0.72  0.77  0.64  110  115  103  17   8  ...  20   1   

Type          FD         +/–.1          
Side atk def all atk def   all atk def  
0      2   1   4   1   3    -1  +1  -2  
1      1   1   4   1   3    -2   0  -2  
2      0   0   6   1   5    -6  -1  -5  
3      6   4  13   7   6    -3  -1  -2  
4      1   0   3   2   1    -2  -1  -1  

[5 rows x 38 columns]

Type      Name Team     R              ACS             K      ... HS%  FK      \
Side      Name Team   all   atk   def  all  atk  def all atk  ... def all atk   
0     crashies  NRG  1.26  1.44  1.08  220  252  188  36  24  ...  22   5   1   
1          s0m  NRG  1.25  1.29  1.23  227  209  253  36  17  ...  41   4   2   
2          FNS  NRG  1.12  1.15  1.10  173  161  186  24  12  ...  17   5   4   
3       ardiis  NRG  1.08  1.27  0.88  190  217  159  27  16  ...  12  10   8   
4       Victor  NRG  0.98  1.05  0.92  172  149  198  29  14  ...  30   6   3   

Type      FD         +/–.1          
Side def all atk def   all atk def  
0      4   4   1   3    +1   0  +1  
1      2   4   2   2     0   0   0  
2      1   0   0   0    +5  +4  +1  
3      2   4   1   3    +6  +7  -1  
4      3   4   2   2    +2  +1  +1  

[5 rows x 38 columns]

Icebox 1:12:59


Type        Name Team     R              ACS             K      ... HS%  FK  \
Side        Name Team   all   atk   def  all  atk  def all atk  ... def all   
0          trexx  KOI  1.24  1.72  0.77  304  391  218  27  18  ...  36   2   
1        sheydos  KOI  1.15  1.56  0.73  193  241  147  17  13  ...  28   1   
2         Wolfen  KOI  0.81  0.81  0.81  168  176  162  14   7  ...  50   5   
3         starxo  KOI  0.73  1.00  0.46  134  149  119  10   6  ...  20   0   
4     koldamenta  KOI  0.59  1.08  0.09   73  134   12   5   5  ...   0   0   

Type          FD         +/–.1          
Side atk def all atk def   all atk def  
0      1   1   1   1   0    +1   0  +1  
1      1   0   2   0   2    -1  +1  -2  
2      4   1   8   5   3    -3  -1  -2  
3      0   0   4   0   4    -4   0  -4  
4      0   0   1   0   1    -1   0  -1  

[5 rows x 38 columns]

...

Type     Name Team     R              ACS             K      ... HS%  FK      \
Side     Name Team   all   atk   def  all  atk  def all atk  ... def all atk   
0       Reita  DFM  0.80  0.79  0.84  263  247  303  15  10  ...  46   2   1   
1     Suggest  DFM  0.75  0.67  0.94  150  107  254  10   5  ...  67   3   2   
2       xnfri  DFM  0.58  0.75  0.17  102  118   65   7   6  ...  20   0   0   
3      Anthem  DFM  0.47  0.59  0.21  128  155   64   6   5  ...  50   0   0   
4     Seoldam  DFM  0.27  0.37  0.05  129  153   74   7   6  ...  27   2   2   

Type      FD         +/–.1          
Side def all atk def   all atk def  
0      1   3   3   0    -1  -2  +1  
1      1   1   1   0    +2  +1  +1  
2      0   1   0   1    -1   0  -1  
3      0   1   1   0    -1  -1   0  
4      0   4   2   2    -2   0  -2  

[5 rows x 38 columns]

Type     Name Team     R              ACS             K      ... HS%  FK      \
Side     Name Team   all   atk   def  all  atk  def all atk  ... def all atk   
0       Cloud  GIA  1.96  2.35  1.80  372  389  366  26   9  ...  37   3   2   
1      nukkye  GIA  1.52  1.30  1.61  216  188  228  12   3  ...  35   1   0   
2       rhyme  GIA  1.45  1.33  1.50  204  204  204  13   3  ...  35   1   0   
3       hoody  GIA  1.20  1.21  1.20  178  199  171  10   4  ...  10   2   1   
4     Fit1nho  GIA  1.01  1.14  0.96  159  168  156  10   3  ...   8   3   0   

Type      FD         +/–.1          
Side def all atk def   all atk def  
0      1   1   0   1    +2  +2   0  
1      1   2   1   1    -1  -1   0  
2      1   1   0   1     0   0   0  
3      1   1   1   0    +1   0  +1  
4      3   2   0   2    +1   0  +1  

[5 rows x 38 columns]

Icebox 51:43


Type     Name Team     R              ACS             K      ... HS%  FK      \
Side     Name Team   all   atk   def  all  atk  def all atk  ... def all atk   
0       takej  DFM  1.00  1.11  0.93  197  200  195  13   6  ...  64   2   2   
1      Anthem  DFM  0.95  1.19  0.79  190  238  159  13   7  ...  41   3   0   
2       Reita  DFM  0.77  0.44  0.99  152   91  194  10   2  ...  73   1   0   
3       xnfri  DFM  0.74  0.57  0.85  200  220  187  12   5  ...  12   1   1   
4     Suggest  DFM  0.69  0.53  0.79  159  169  153  11   5  ...  17   1   0   

Type      FD         +/–.1          
Side def all atk def   all atk def  
0      0   1   0   1    +1  +2  -1  
1      3   1   1   0    +2  -1  +3  
2      1   5   3   2    -4  -3  -1  
3      0   1   0   1     0  +1  -1  
4      1   4   1   3    -3  -1  -2  

[5 rows x 38 columns]

Type     Name Team     R              ACS             K      ... HS%  FK      \
Side     Name Team   all   atk   def  all  atk  def all atk  ... def all atk   
0      nukkye  GIA  1.30  1.24  1.37  249  237  269  20  12  ...  35   4   1   
1       hoody  GIA  1.30  1.10  1.59  238  203  291  17   9  ...  28   3   2   
2       rhyme  GIA  1.20  1.22  1.17  198  170  240  15   8  ...  35   2   1   
3       Cloud  GIA  1.19  1.06  1.39  202  193  217  16   9  ...  67   1   1   
4     Fit1nho  GIA  0.93  1.22  0.50  162  247   37  10   9  ...  50   2   2   

Type      FD         +/–.1          
Side def all atk def   all atk def  
0      3   2   1   1    +2   0  +2  
1      1   1   0   1    +2  +2   0  
2      1   3   2   1    -1  -1   0  
3      0   0   0   0    +1  +1   0  
4      0   2   2   0     0   0   0  

[5 rows x 38 columns]

# OOP Design

In [ ]:
class Event:
    def __init__(self,url):
        self.url = url
        # self.dates -> str(maybe pd.datatime)
        # self.players -> List[Player]
        # self.prize_pool -> int,float
        # self.region -> str
        # self.matches -> List[Match]

In [ ]:
class Match:
    def __init__(self,html):
        # self.date -> str
        # self.stage -> str
        # self.maps -> List[Map]
        # self.winner -> Team
        